In [1]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1,0,2" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

Using TensorFlow backend.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

In [3]:
root_dir = 'data/tree-rep-profiles_o2o/'
train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_rep_ans.shape)
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

test of sent2vec vector: (506, 216, 768) (506, 216) (506, 216, 1)


In [4]:
def _shuffle(X, X2 ,X3):
#     X3 = np.take(train_fam_ans,[0],axis=-1) #只train第幾個familiy
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize])

In [5]:
train_emb, train_emb_api, train_rep_ans = _shuffle(train_emb, train_emb_api, train_rep_ans)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_rep_ans.shape)

train of sent2vec vector: (7103, 216, 768) (7103, 216) (7103, 216, 1)
valid of sent2vec vector: (451, 216, 768) (451, 216) (451, 216, 1)


## Model Hyper-Parameters

In [6]:
emb_dim = valid_emb.shape[-1] #被除數
num_heads = 48#除數，要整除
max_length = test_emb_api.shape[1] # max sequence length
# fam_num = train_fam_ans.shape[1]
vocabulary_size = emb_matrix.shape[0]-1
transformer_depth = 1
transformer_dropout = 0.1
l2_reg_penalty = 1e-6#1e-4
dp_rate = 0.01

traina = True #改
batch_size = 128 #改

In [7]:
constrain = keras.constraints.MinMaxNorm(min_value=0.0, max_value=0.0, rate=1.0, axis=0)
init = keras.initializers.Ones()
coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding') #positional encoding
act_layer = TransformerACT(
            name='adaptive_computation_time')

transformer_block = TransformerBlock(
            name='transformer', num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            # Allow bi-directional attention
            use_masking=False)
add_segment_layer = Add(name='add_segment')
l2_regularizer = (regularizers.l2(l2_reg_penalty) if l2_reg_penalty else None)

## Model Archeitecture

In [8]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
#shape=(max_length,emb_dim),,batch_shape=(batch_size,max_length,emb_dim)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
#shape=(max_length,),batch_shape=(batch_size,max_length)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length
                    ,trainable=True,name='api_emb')(sent_ids) #改

segment_embeddings = Add()([sentemb,api_emb])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
att_rnn = GRU(int(emb_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(segment_embeddings)
self_att = SeqSelfAttention(attention_activation='sigmoid',name='self_attention')(att_rnn)
# self_att = MultiHead(SeqSelfAttention(attention_activation='sigmoid',name='self_attention'),layer_num=2,name='Multi_Head')(att_rnn)
# multi_head = Reshape((max_length,-1))(self_att)
# multi_head = Lambda(lambda x: keras.backend.concatenate(x,axis=-2),name='reshape')(self_att)
rep_prediction = (
        TimeDistributed(Dense(1, name='0_1_predict', activation='sigmoid',trainable=True),name='out_rep') # hard_sigmoid
    (self_att))

In [10]:
model = Model(inputs=[sent_ids1,sentemb1], outputs=[rep_prediction]) #out
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 216)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 216, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 216)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 216, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [11]:
model.load_weights('./model/o2o_130_gru_selfatt/byterep_130fam_0701_gruatt_sent2vec.h5',by_name=True) #改

model loss opt

In [12]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
from keras.metrics import top_k_categorical_accuracy
def custom_acc1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred,k=3)
from keras.metrics import binary_accuracy
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

def precision(y_true, y_pred):
    return tf.keras.metrics.Precision(y_true,y_pred)[1]
def recall(y_true, y_pred):
    return tf.keras.metrics.Recall(y_true,y_pred)[1]
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))
def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))
# from sklearn.metrics import f1_score
# def f1_sk(y_true,y_pred):
#     score = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
#     return score

# 訓練參數
los = [hn_multilabel_loss,binary_focal_loss(alpha=.25, gamma=2)] # 1st stage.  f1_loss
#SINGLE
# los = [binary_focal_loss(alpha=.25, gamma=2)] #改
# los = [f1_loss]
# los = [hn_multilabel_loss]
los = [losses.binary_crossentropy]
# MML
'''los = []
for i in range(fam_num):
    los.append(binary_focal_loss(alpha=.25, gamma=2))
los = [losses.binary_crossentropy] + los'''


metric = {'out_rep': bin_acc,'family': f1_metric} # 1st stage. km.f1_score()
#SINGLE
metric = [f1_metric,bin_acc]
# metric = [km.f1_score(),bin_acc,km.binary_f1_score()]
# metric = {'RasMMA': 'acc'}
metric = [bin_acc] #改
#MML
'''metrics = []
for i in range(fam_num+1):
    metrics.append('acc')
# metrics = {}
# metrics['RasMMA'] = 'acc'
# for i in range(fam_num):
#     metrics['fam'+str(i)]='acc'
metric = metrics'''


loss_weight = [1,1] #stage1 0.95,0.05  #1st stage # 2nd stage [0.01,0.99]
#SINGLE
loss_weight = [1]
#MML
'''loss_weight = []
for i in range(fam_num):
    loss_weight.append(0.95)
loss_weight = [0.05] + loss_weight'''

learning_rate = 5e-4#2e-4 # 2nd stage: 1e-4 @1st:2e-4 0.002
# batch_size = 128 #32 #128

num_epochs = 1000
patien = 50

model_save_path = './model/o2o_130_gru_selfatt/byterep_130fam_0701_gruatt_sent2vec.h5' #改
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0]

In [13]:
# import tensorflow as tf
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False) #clipnorm=1. , clipvalue=1.
optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.) #改
# tf.keras.optimizers.Nadam
lr_scheduler1 = callbacks.LearningRateScheduler(
        CosineLRSchedule(lr_high=0.0015, lr_low=1e-8, #learning_rate #改
                         initial_period=num_epochs),
        verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/3),
                                      min_lr=1e-8,mode='min')

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)#{'word_predictions': masked_perplexity})

In [14]:
# save best, early stopping, 2 models ens weight:(best=0.8,last=0.2)
history = History()
stop_nan = callbacks.TerminateOnNaN()
model_callbacks = [
        callbacks.ModelCheckpoint(
            model_save_path, #val_f1_metric,max。val_family_f1_metric
            monitor='val_loss',mode='min' ,save_best_only=True, verbose=1,save_weights_only=True), #改
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
        lr_scheduler, lr_scheduler1,history,stop_nan
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

In [15]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1):#, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = class_prediction: shape為(N, max_seq_length, 1)
    Y_train2 = family_prediction(stage2): shape為(N, fam_num)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
#             train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
#             yield(train_X2,train_Y2)
#             yield ([train_X1,train_X2],[train_Y1,train_Y2]) #ori
            yield ([train_X1,train_X2],[train_Y1]) #改
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break
            
#     data_size = X_train.shape[0]
#     ep = data_size / batch_size

In [16]:

H = model.fit_generator( #train_rep_ans
    generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_rep_ans)#,Y_train2=train_fam_ans) #Y_train2
#                     generator=training_batch(batch_size=batch_size,X_train1=valid_emb_api,X_train2=valid_emb ,
#                                              Y_train1=train_rep_ans,Y_train2=train_fam_ans)
                        , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]) #ori
#                    ,validation_data= (valid_emb, valid_fam_ans) 
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]) #ori #改
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2) #validY2
                    ,max_queue_size=10 #,class_weight=fam_weights#[None,fam_weights] #改
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")
#1st:train 0_1_prediction=0.14XX

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/home/leoqaz12/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.0015.
56/56 [==============================] - 28s 495ms/step - loss: 0.5923 - bin_acc: 0.6961 - val_loss: 0.5799 - val_bin_acc: 0.7176

Epoch 00001: val_loss improved from inf to 0.57985, saving model to ./model/o2o_130_gru_selfatt/byterep_130fam_0701_gruatt_sent2vec.h5
Epoch 2/1000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0014999962989260677.
56/56 [==============================] - 23s 419ms/step - loss: 0.5530 - bin_acc: 0.7259 - val_loss: 0.5224 - val_bin_acc: 0.7585

Epoch 00002: val_loss improved from 0.57985 to 0.52239, saving model to ./model/o2o_130_gru_selfatt/byterep_130fam_0701_gruatt_sent2vec.h5
Epoch 3/1000

Epoch 00003: LearningRateScheduler setting learning rate to 0.0014999851957407985.
56/56 [==============================] - 23s 412ms/step - loss: 0.5222 - bin_acc: 0.7609 - val_loss: 0.5200 - val_bin_acc: 0.7491

Epoch 00003: val_loss improved from 0.52239 to 0.52003, s

56/56 [==============================] - 24s 420ms/step - loss: 0.3815 - bin_acc: 0.8388 - val_loss: 0.3675 - val_bin_acc: 0.8466

Epoch 00054: val_loss did not improve from 0.35649
Epoch 55/1000

Epoch 00055: LearningRateScheduler setting learning rate to 0.0014892335181387727.
56/56 [==============================] - 24s 423ms/step - loss: 0.3620 - bin_acc: 0.8377 - val_loss: 0.3751 - val_bin_acc: 0.8414

Epoch 00055: val_loss did not improve from 0.35649
Epoch 56/1000

Epoch 00056: LearningRateScheduler setting learning rate to 0.0014888320690694496.
56/56 [==============================] - 23s 418ms/step - loss: 0.4010 - bin_acc: 0.8229 - val_loss: 0.3589 - val_bin_acc: 0.8520

Epoch 00056: val_loss did not improve from 0.35649
Epoch 57/1000

Epoch 00057: LearningRateScheduler setting learning rate to 0.0014884233280752315.
56/56 [==============================] - 23s 419ms/step - loss: 0.3838 - bin_acc: 0.8348 - val_loss: 0.3681 - val_bin_acc: 0.8471

Epoch 00057: val_loss did not

56/56 [==============================] - 24s 433ms/step - loss: 0.3768 - bin_acc: 0.8335 - val_loss: 0.3762 - val_bin_acc: 0.8512

Epoch 00083: val_loss did not improve from 0.34010
Epoch 84/1000

Epoch 00084: LearningRateScheduler setting learning rate to 0.0014746474174796296.
56/56 [==============================] - 23s 419ms/step - loss: 0.3654 - bin_acc: 0.8472 - val_loss: 0.3373 - val_bin_acc: 0.8632

Epoch 00084: val_loss improved from 0.34010 to 0.33725, saving model to ./model/o2o_130_gru_selfatt/byterep_130fam_0701_gruatt_sent2vec.h5
Epoch 85/1000

Epoch 00085: LearningRateScheduler setting learning rate to 0.0014740364022167614.
56/56 [==============================] - 23s 419ms/step - loss: 0.3657 - bin_acc: 0.8401 - val_loss: 0.3486 - val_bin_acc: 0.8553

Epoch 00085: val_loss did not improve from 0.33725
Epoch 86/1000

Epoch 00086: LearningRateScheduler setting learning rate to 0.0014734182410562563.
56/56 [==============================] - 24s 422ms/step - loss: 0.3543 -

56/56 [==============================] - 23s 417ms/step - loss: 0.3617 - bin_acc: 0.8419 - val_loss: 0.3895 - val_bin_acc: 0.8321

Epoch 00112: val_loss did not improve from 0.32864
Epoch 113/1000

Epoch 00113: LearningRateScheduler setting learning rate to 0.0014540506995711173.
56/56 [==============================] - 24s 420ms/step - loss: 0.3507 - bin_acc: 0.8438 - val_loss: 0.3491 - val_bin_acc: 0.8573

Epoch 00113: val_loss did not improve from 0.32864
Epoch 114/1000

Epoch 00114: LearningRateScheduler setting learning rate to 0.0014532351862404302.
56/56 [==============================] - 24s 422ms/step - loss: 0.3712 - bin_acc: 0.8350 - val_loss: 0.3697 - val_bin_acc: 0.8477

Epoch 00114: val_loss did not improve from 0.32864
Epoch 115/1000

Epoch 00115: LearningRateScheduler setting learning rate to 0.0014524127323117108.
56/56 [==============================] - 23s 417ms/step - loss: 0.3503 - bin_acc: 0.8434 - val_loss: 0.3549 - val_bin_acc: 0.8638

Epoch 00115: val_loss did 

56/56 [==============================] - 23s 419ms/step - loss: 0.3881 - bin_acc: 0.8340 - val_loss: 0.3587 - val_bin_acc: 0.8477

Epoch 00141: val_loss did not improve from 0.31530
Epoch 142/1000

Epoch 00142: LearningRateScheduler setting learning rate to 0.001427614205899954.
56/56 [==============================] - 23s 419ms/step - loss: 0.3186 - bin_acc: 0.8669 - val_loss: 0.3307 - val_bin_acc: 0.8619

Epoch 00142: val_loss did not improve from 0.31530
Epoch 143/1000

Epoch 00143: LearningRateScheduler setting learning rate to 0.0014266009588563553.
56/56 [==============================] - 24s 424ms/step - loss: 0.3803 - bin_acc: 0.8346 - val_loss: 0.3751 - val_bin_acc: 0.8219

Epoch 00143: val_loss did not improve from 0.31530
Epoch 144/1000

Epoch 00144: LearningRateScheduler setting learning rate to 0.0014255810340837954.
56/56 [==============================] - 23s 419ms/step - loss: 0.3707 - bin_acc: 0.8362 - val_loss: 0.3480 - val_bin_acc: 0.8651

Epoch 00144: val_loss did n

56/56 [==============================] - 23s 419ms/step - loss: 0.3501 - bin_acc: 0.8463 - val_loss: 0.3636 - val_bin_acc: 0.8239

Epoch 00170: val_loss did not improve from 0.31489
Epoch 171/1000

Epoch 00171: LearningRateScheduler setting learning rate to 0.0013955572165428227.
56/56 [==============================] - 24s 421ms/step - loss: 0.3486 - bin_acc: 0.8459 - val_loss: 0.3200 - val_bin_acc: 0.8774

Epoch 00171: val_loss did not improve from 0.31489
Epoch 172/1000

Epoch 00172: LearningRateScheduler setting learning rate to 0.001394354640262798.
56/56 [==============================] - 23s 418ms/step - loss: 0.3640 - bin_acc: 0.8478 - val_loss: 0.3163 - val_bin_acc: 0.8766

Epoch 00172: val_loss did not improve from 0.31489
Epoch 173/1000

Epoch 00173: LearningRateScheduler setting learning rate to 0.0013931457045119583.
56/56 [==============================] - 24s 421ms/step - loss: 0.3733 - bin_acc: 0.8371 - val_loss: 0.3330 - val_bin_acc: 0.8683

Epoch 00173: val_loss did n

56/56 [==============================] - 23s 419ms/step - loss: 0.3648 - bin_acc: 0.8432 - val_loss: 0.3524 - val_bin_acc: 0.8665

Epoch 00232: val_loss did not improve from 0.30943
Epoch 233/1000

Epoch 00233: LearningRateScheduler setting learning rate to 0.0013094571293624097.
56/56 [==============================] - 24s 421ms/step - loss: 0.3812 - bin_acc: 0.8328 - val_loss: 0.3533 - val_bin_acc: 0.8535

Epoch 00233: val_loss did not improve from 0.30943
Epoch 234/1000

Epoch 00234: LearningRateScheduler setting learning rate to 0.0013078851281296197.
56/56 [==============================] - 23s 418ms/step - loss: 0.3453 - bin_acc: 0.8549 - val_loss: 0.3342 - val_bin_acc: 0.8644

Epoch 00234: val_loss did not improve from 0.30943
Epoch 235/1000

Epoch 00235: LearningRateScheduler setting learning rate to 0.0013063076208451906.
56/56 [==============================] - 23s 418ms/step - loss: 0.3159 - bin_acc: 0.8684 - val_loss: 0.3429 - val_bin_acc: 0.8647

Epoch 00235: val_loss did 

Epoch 00002: LearningRateScheduler setting learning rate to 0.0014999962989260677.
56/56 [==============================] - 23s 419ms/step - loss: 0.5530 - bin_acc: 0.7259 - val_loss: 0.5224 - val_bin_acc: 0.7585


# Evaluate

In [17]:
model.load_weights(model_save_path)
model.summary()

In [21]:
from keras.models import model_from_json, model_from_yaml
from sklearn.metrics import f1_score,recall_score,precision_score,hamming_loss , roc_auc_score

pickle.dump(file=open(tensorboard_log_path + '/'+'byterep_gruatt_H2.pkl','wb'),obj=H) 
json_string = model.to_json()
yaml_string = model.to_yaml()
pickle.dump(file=open(tensorboard_log_path + '/'+'byterep_gruatt_arch.pkl','wb'),obj=(json_string,yaml_string))

In [29]:
ans_t = model.predict([test_emb_api,test_emb])
ans_v = model.predict([valid_emb_api,valid_emb])

In [24]:
y_true.shape

(506, 216, 1)

In [25]:
ans_t.shape

(506, 216, 1)

In [30]:
y_true = test_rep_ans.squeeze(axis=-1)
ans_t = ans_t.squeeze(axis=-1)
area_score_t = roc_auc_score(y_true,ans_t,average='micro')
y_true = valid_rep_ans.squeeze(axis=-1)
ans_v = ans_v.squeeze(axis=-1)
area_score_v = roc_auc_score(y_true,ans_v,average='micro')
print('valid roc auc score:',area_score_v,'test:',area_score_t)

valid roc auc score: 0.894314815832789 test: 0.8725151558369375


In [36]:
score_list = []
threashold_list = []
max_score = 0
for i in range(1,1000):
    thr = i/1000
    final_ans_t = []
    for sample in ans_t:
        sample_ans = []
        for value in sample:
            if value < thr: #0.26 #0.33
                sample_ans.append(0)
            else:
                sample_ans.append(1)
        final_ans_t.append(sample_ans)
    final_ans_t = np.array(final_ans_t)
#     print(final_ans_t.shape , sum(final_ans_t[0]))

    final_ans_v = []
    for sample in ans_v:
        sample_ans = []
        for value in sample:
            if value < thr: #0.26 #0.33
                sample_ans.append(0)
            else:
                sample_ans.append(1)
        final_ans_v.append(sample_ans)
    final_ans_v = np.array(final_ans_v)
#     print(final_ans_v.shape , sum(final_ans_v[0]))

    y_true = test_rep_ans.squeeze(axis=-1)
    recall_t = recall_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    precision_t = precision_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    f1_t = f1_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    h_loss_t = hamming_loss(y_true,final_ans_t)
#     print('Test: (recall/precision/f1/h_loss)',recall_t ,precision_t, f1_t , h_loss_t)

    y_true = valid_rep_ans.squeeze(axis=-1)
    recall_v = recall_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    precision_v = precision_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    f1_v = f1_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    h_loss_v = hamming_loss(y_true,final_ans_v)
    score = f1_v+f1_t-h_loss_v-h_loss_t
    if score>max_score:
        print('Best choice threashold now is:',thr,f1_v,f1_t,h_loss_v,h_loss_t)
        max_score = score
    score_list.append(score)
    threashold_list.append(thr)
#     print('Valid: (recall/precision/f1/h_loss)=>',recall_v ,precision_v, f1_v , h_loss_v)
print(max_score)

Best choice threashold now is: 0.5 0.8819430841984498 0.8658494218075338 0.043309107333497576 0.04946201141853316
Best choice threashold now is: 0.52 0.8817946990116802 0.8657549261390528 0.04321672004598834 0.049306470502122675
1.655026434602622


Best choice threashold now is: 0.5 0.8819430841984498 0.8658494218075338 0.043309107333497576 0.04946201141853316
Best choice threashold now is: 0.52 0.8817946990116802 0.8657549261390528 0.04321672004598834 0.049306470502122675
1.655026434602622

In [107]:
def chk_assetion(root_dir,ans_d,pad_num=0.49756694,max_pad=0.49756694,min_pad=0.49756693): #改treshold
    '''
    檢查profile順序是否與embedding順序一致
    Input: root_dir
    Output: length list
    '''
    dev_length = []
    dev_path_li = []
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir +fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
#             dev_path_li.extend(hl_list)
            for hl_f in hl_list:
                with open(hl_f,encoding='ISO 8859-1') as f: #X2
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
                lines = lines.splitlines()
                dev_length.append(len(lines))
                dev_path_li.append(hl_f)
            
    for ans,length in zip(ans_d,dev_length):
        assert ans[:length][-1] != pad_num
        assert ans[:length+1][-1] < max_pad
        assert ans[:length+1][-1] > min_pad
    return dev_path_li

In [108]:
valid_path_li = chk_assetion('./data/tree-rep-profiles_o2o/DEV/',ans_v)
test_path_li = chk_assetion('./data/tree-rep-profiles_o2o/TEST/',ans_t)

100%|██████████| 45/45 [00:00<00:00, 223.95it/s]


In [109]:
(test_path_li)

['./data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G395/44fe487808dd8fc0ff048d416be41a9414b66003d15cd6f0d75251f921c537e1_3416.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G395/88a899e31e4d89952f807b9ee2f69c3344e2d0137976b4988e5e95d2c7a95333_3236.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G394/413876356b48a780ee82c4452a511d81065e84fc69b748c0752eb3064781d5b8_3280.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G394/0c1ce1ef6a41b78c59aa4a3096c8777c70ba06cfe102046fc6b7a9617a72cad3_3320.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G397/01519c4765be0d99fce9a9bb4aa338793ab4742294a63f16540c05b34f74c175_2936.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G397/6f3460e84dbbe9d2d1065f58b57e5ac048fe34584fe2a17b5909919013301cef_3404.profile',
 './data/tree-rep-profiles_o2o/TEST/85.ipamor_0.8/G102/6e19cb71a89604d4b65c48fdc70d7fb2d39c7e835eeb60f860f535ecdd3af813_3368.profile',
 './data/tree-rep-profiles_o2o/TEST/85.ipamor_0.8/G102/

In [ ]:
def get_all_add_emb(dev_emb_api,dev_emb,model=model):
    intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.layers[5].output)
    intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
    print('Emb shape:',intermediate_output.shape)
    return intermediate_output

In [111]:

intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.layers[5].output)
intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
intermediate_output.shape

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 216)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 216, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 216)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 216, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

(451, 216, 768)

In [114]:
intermediate_output[0][10]

array([ 6.2618005e-01, -2.1089106e+00, -3.0442843e+00,  5.3742480e-01,
        3.0402291e+00,  1.4082639e+00, -3.9007258e-01, -1.2419176e+00,
       -3.1372221e+00, -3.2730191e+00,  5.5726153e-01, -5.0289059e-01,
        1.0976368e+00, -1.1940066e+00, -2.0536981e+00,  2.4708271e+00,
        2.3215792e+00,  3.9515114e+00,  1.9344985e+00, -2.2881236e+00,
        2.2857480e+00, -2.0793555e+00,  1.4051611e+00,  3.4353054e+00,
       -4.0642748e+00, -1.7742591e+00, -9.2830986e-01, -3.3688855e+00,
       -1.1665313e+00, -1.2051904e+00, -2.6467364e+00,  1.9879489e+00,
        2.2966270e+00, -7.6591301e-01, -1.3570929e+00,  1.1703807e+00,
       -3.3427026e+00, -2.9934566e+00, -1.3248649e-01,  2.1944325e+00,
        2.6791778e-01,  4.2409101e-01, -9.6809852e-01, -1.5632174e+00,
       -4.7623760e-01,  5.6784534e+00,  2.5199026e-01, -6.0443032e-01,
        1.0090823e+00,  3.4446137e+00, -6.5510547e-01, -3.1890869e-03,
       -3.0715849e+00, -5.2878947e+00, -1.7584171e+00, -1.5329256e+00,
      

In [64]:
dev_length = []
root_dir = './data/tree-rep-profiles_o2o/DEV/'
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir +fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for hl_f in hl_list:
            with open(hl_f,encoding='ISO 8859-1') as f: #X2
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
            lines = lines.splitlines()
            dev_length.append(len(lines))
#             print(len(lines))
#             break
test_length = []
root_dir = './data/tree-rep-profiles_o2o/TEST/'
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir +fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for hl_f in hl_list:
            with open(hl_f,encoding='ISO 8859-1') as f: #X2
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
            lines = lines.splitlines()
            test_length.append(len(lines))

100%|██████████| 45/45 [00:00<00:00, 235.73it/s]


In [98]:
for ans,length in zip(ans_v,dev_length):
    assert ans[:length][-1] != 0.49756694
    assert ans[:length+1][-1] < 0.49756694
    assert ans[:length+1][-1] > 0.49756693

In [96]:
ans[:length+1][-1] > 0.49756693

True

In [83]:
ans[:105][-1]

0.49756694

In [69]:
ans_t[-1][:200][-1]

0.49756694